In [10]:
from CheckEnvAgent import CreateFolder, load_module_player, add_game_to_syspath, setup_game, save_json
import itertools
import warnings 
from setup import SHOT_PATH
import numpy as np
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)


In [17]:
import itertools
env_name = 'Splendor_v2'
list_name_agent = ['test', 'An', 'FusionwideNNAlgorithm', 'Trang', 'Chain3Algorithm']
add_game_to_syspath(env_name)
__env__ = setup_game(env_name)
dict_level = {}
combinations = []
dict_result = {}
for agent in list_name_agent:
    list_name_agent_copy = list_name_agent.copy()
    list_name_agent_copy.remove(agent)
    dict_level[env_name] = {}
    p0 = load_module_player(agent).Test
    combis = enumerate(itertools.combinations(list_name_agent_copy, __env__.getAgentSize()-1))

    level_max = 1
    for level0, combination in combis:
        list_level = [0, 0, list(combination)]
        dict_level[env_name][str(level0+1)] = list_level
        level_max += 1
    
    save_json(f'{SHOT_PATH}Log/level_game{env_name}.json', dict_level)
    per_player = list(np.load(f'{SHOT_PATH}Agent/{agent}/Data/{env_name}_0/Train.npy',allow_pickle=True))
    win_total_player = 0
    for level in range(level_max):
        win, per = __env__.numba_main_2(p0, 100, per_player, level)
        win_total_player += win
        print(level, win)
    print(agent, win_total_player)
    dict_result[agent] = win_total_player
print(dict_result)

0 21
1 26
2 8
3 29
4 11
test 95
0 14
1 15
2 15
3 13
4 11
An 68
0 46
1 68
2 42
3 44
4 54
FusionwideNNAlgorithm 254
0 2
1 4
2 3
3 0
4 1
Trang 10
0 52
1 50
2 64
3 37
4 46
Chain3Algorithm 249
{'test': 95, 'An': 68, 'FusionwideNNAlgorithm': 254, 'Trang': 10, 'Chain3Algorithm': 249}
